##IMPORTS

In [15]:
import tensorflow as tf
import pandas as pd
import re

In [16]:
dataset = open("corpus.txt", "r", encoding = "utf-8-sig").read()

In [17]:
dataset[:100]

'In the Devoid of Sorrow\nThe Air and my Heart full of Luster\nBehind the Mountains\nThere must be a key'

##PREPROCESSING

In [34]:
def clean_text(text):
    # lower-case
    text = text.lower()

    # clean archaisms and other special cases
    text = re.sub(r"'cause ", "because", text)
    text = re.sub(r"shalt", "shall", text)
    text = re.sub(r"kaos", "chaos", text)
    text = re.sub(r"'till", "until", text)
    text = re.sub(r"fuckin'", "fucking", text)
    text = re.sub(r"couldst", "could", text)
    text = re.sub(r"sayeth", "says", text)
    text = re.sub(r"calleth", "calls", text)
    text = re.sub(r"&", " and ", text)
    text = re.sub(r"sathan", "satan", text)
    text = re.sub(r"'em", "them", text)

    # possible parts of other words
    text = re.sub(r"(?<![a-zA-Z])thy(?![a-zA-Z])|thine", "your", text)
    text = re.sub(r"thou|thee|(?<![a-zA-Z])ye(?![a-zA-Z])", "you", text)
    text = re.sub(r"(?<![a-zA-Z])o(?![a-zA-Z])", "oh", text)
    text = re.sub(r"(?<![a-zA-Z])'tis(?![a-zA-Z])", "it is", text)
    text = re.sub(r"(?<![a-zA-Z])thru(?![a-zA-Z])", "through", text)
    text = re.sub(r"(?<![a-zA-Z])ov(?![a-zA-Z])", "of", text)
    text = re.sub(r"(?<![a-zA-Z])hast(?![a-zA-Z])", "have", text)
    
    # punctuation, except: 
    # \n
    text = re.sub(r"[^\n&a-zA-Z']", " ", text)

    # double whitespaces
    text = re.sub(r" +", " ", text)


  
    return text

In [35]:
text = clean_text(dataset)

In [36]:
text

"in the devoid of sorrow\nthe air and my heart full of luster\nbehind the mountains\nthere must be a key to the gates\nright passed easy talk\nwe can once again walk the path of sorrow\nlet the turn of search bring you away\ntwo shades of the midnight fire\nin the night sky\nyou waiting child walk with me\nwalk by side of fall\nthe essence to open those gates\nand still we must walk the path of sorrow\nthe secret lights in the sky\nvision\nall one\nit seems\nthe blades are sharpened\nthey will all learn\nyes destroy your dream\nyour desert of dreams\nit's undeserved\nfor dark medieval times\nhere upon the hill\nyou all find wisdom\nwaiting for a lot of\nendless screaming\ndesert of dreams\neternity in your dreams\nhere you are\ndark medieval times\ninto the mighty forest\nas i walk\nwith a spirit\ninto the binding forest\nin fog and darkness\ni walk\ni disappear\ninto the mighty forest\nit's beyond the sweet\nand tainted landscapes\nthe shadow walking\nyour sanctity\nsought the story\n

In [37]:
tokens = re.findall(r"\S+|\n", text)

In [38]:
tokens[600:666]

['we',
 'burnt',
 'them',
 'in',
 'the',
 'purgatory',
 'them',
 'the',
 'children',
 'of',
 'god',
 '\n',
 'barely',
 'forgotten',
 'these',
 'times',
 'are',
 '\n',
 'but',
 'not',
 'for',
 'a',
 'soul',
 'whose',
 'rest',
 "hasn't",
 'been',
 'found',
 '\n',
 'from',
 'beyond',
 'come',
 'the',
 'storms',
 '\n',
 'landscapes',
 'turn',
 'to',
 'ash',
 'before',
 'my',
 'pressured',
 'eyes',
 '\n',
 'nothingness',
 'turns',
 'to',
 'nothingness',
 '\n',
 'and',
 'my',
 'imagination',
 'fades',
 'like',
 'dust',
 'clouds',
 'over',
 '\n',
 'this',
 'deserted',
 'land',
 'feel',
 'to',
 'now',
 'satyricon',
 '\n']

##EXPLORATORY DATA ANALYSIS

In [39]:
# total tokens
len(tokens)

391756

In [40]:
# unique tokens
len(set(tokens))

17600

In [ ]:
# hapax legomena - outliers - possibly remove?

In [ ]:
# vocab analysis - lemmatize

##SEQUENCES

In [41]:
seq_len = 50
X_items = []
y = []

for i in range(0, len(tokens)-seq_len):
	X_items.append(tokens[i:i+seq_len])
	y.append(tokens[i+seq_len])

In [42]:
X = [" ".join(item) for item in X_items]

In [43]:
X[:5]

['in the devoid of sorrow \n the air and my heart full of luster \n behind the mountains \n there must be a key to the gates \n right passed easy talk \n we can once again walk the path of sorrow \n let the turn of search bring you',
 'the devoid of sorrow \n the air and my heart full of luster \n behind the mountains \n there must be a key to the gates \n right passed easy talk \n we can once again walk the path of sorrow \n let the turn of search bring you away',
 'devoid of sorrow \n the air and my heart full of luster \n behind the mountains \n there must be a key to the gates \n right passed easy talk \n we can once again walk the path of sorrow \n let the turn of search bring you away \n',
 'of sorrow \n the air and my heart full of luster \n behind the mountains \n there must be a key to the gates \n right passed easy talk \n we can once again walk the path of sorrow \n let the turn of search bring you away \n two',
 'sorrow \n the air and my heart full of luster \n behind the m

In [44]:
y[:5]

['away', '\n', 'two', 'shades', 'of']

In [ ]:
# TESTING THE DIMENSIONS

In [45]:
x_test = [re.findall(r"\S+|\n", sequence) for sequence in X]

In [46]:
lengths_x = [len(item) for item in x_test]

In [47]:
set(lengths_x)

{50}

In [48]:
y_test = [re.findall(r"\S+|\n", sequence) for sequence in y]

In [49]:
lengths_y = [len(item) for item in y_test]

In [50]:
set(lengths_y)

{1}

##TOKENIZATION

In [51]:
from keras.preprocessing.text import Tokenizer

# to keep the \n
filters_ = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t' 
tokenizer = Tokenizer(filters=filters_)

tokenizer.fit_on_texts(X)

In [52]:
X_emb = tokenizer.texts_to_sequences(X)
y_emb = tokenizer.texts_to_sequences(y)

In [ ]:
# TESTING THE DIMENSIONS

In [53]:
lengths_X_emb = [len(item) for item in X_emb]

In [54]:
set(lengths_X_emb)

{50}

In [55]:
lengths_y_emb = [len(item) for item in y_emb]

In [56]:
set(lengths_y_emb)

{1}

In [ ]:
from tensorflow.keras.utils import to_categorical

vocab_size = len(tokenizer.word_index)

y_categorical = to_categorical(y_emb, num_classes=vocab_size)

##MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Embedding, LSTM, Dense, Dropout

In [ ]:
INPUT_SHAPE = seq_len
VOCABULARY_SIZE = len(tokenizer.word_index)
EMBEDDING_DIMENSION = 50
LSTM_UNITS = 50
DROPOUT_RATE = 0.2

In [ ]:
rnn = Sequential()

rnn.add(Input(INPUT_SHAPE,))
# +1 here resolves the indexing problem during training
rnn.add(Embedding(VOCABULARY_SIZE+1, EMBEDDING_DIMENSION))
rnn.add(LSTM(LSTM_UNITS, return_sequences=True))
rnn.add(Dense(VOCABULARY_SIZE, activation="softmax"))

In [ ]:
rnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
rnn.summary()

##MODEL TRAINING

##GENERATION